In [2]:
import pandas as pd
import numpy as np


In [3]:
tweets=pd.read_csv('Tweets.csv')

In [4]:
tweets.tail()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,2015-02-22 11:58:51 -0800,"dallas, TX",NaN


In [5]:
text_data=tweets['text'].apply(lambda x: str(x))
text_label=tweets.airline_sentiment

In [106]:
# map sentiment to ordinal values
sentiment_map={'positive':1, 'neutral':0, 'negative':-1}
response=text_label.map(sentiment_map)

In [6]:
import spacy
nlp = spacy.load('en')

In [8]:
#gets the average wordvec
vec1 = [doc.vector for doc in nlp.pipe(text_data, n_threads=50)]
tweets['word_vec'] = list(np.array(vec1))

In [9]:
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,word_vec
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),"[-0.0611154, 0.304086, -0.097632, -0.201844, 0..."
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),"[-0.0165897, 0.14309, -0.137645, -0.164577, 0...."
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),"[0.0355374, 0.173601, -0.17913, -0.0846395, 0...."
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),"[-0.111944, 0.260451, -0.0748006, -0.0820337, ..."
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),"[-0.174232, 0.280287, -0.158224, -0.0262064, 0..."


# set up model 

In [42]:
#reformat to the proper matrix dimension (num. samples, vector dimension of 300)
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

In [131]:
X_train=csr_matrix(list(tweets['word_vec']))
X_train.shape

(14640, 300)

In [115]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


In [121]:
def model_train(X, model_name):

    if model_name =='SGDClassifier':
        model = SGDClassifier(loss='log', penalty='l2',alpha=1e-4, n_iter=40, random_state=42,shuffle=True)
        
    if model_name=='RandomForestClassifier':
        model =RandomForestClassifier(n_estimators=600, max_depth=6, random_state=123) 
        
    if model_name=='MLP':
        model = MLPClassifier(alpha=1e-4, hidden_layer_sizes=(10,5), random_state=12,validation_fraction=0)

    y = response

    from sklearn.cross_validation import StratifiedKFold
    skf = StratifiedKFold(y, n_folds=5)

    fold=0
    accuracy=[]
    ypred=[]
    for train_index, test_index in skf:
        fold+=1
        print (fold)
        X_training, X_test = X[train_index], X[test_index]
        y_training, y_test = y[train_index], y[test_index]
        model.fit(X_training,y_training)
        pred=model.predict(X_test)        
        val_pred=pd.DataFrame(pred, index=test_index)
        ypred.append(val_pred)
        cal_error=np.mean(y_test==pred)
        accuracy.append(cal_error)
    return print('Mean accuracy from 5-fold validation: {}' .format(np.mean(accuracy)))

In [111]:
model_train(X_train, 'SGDClassifier') 

1
2
3
4
5
Mean accuracy from 5-fold validation: 0.7924848996671422


In [122]:
model_train(X_train, 'RandomForestClassifier') 

1
2
3
4
5
Mean accuracy from 5-fold validation: 0.7083340721048996


In [120]:
model_train(X_train, 'MLP')

1
2
3
4
5
Mean accuracy from 5-fold validation: 0.7998649137025452


# train the MLP model, which performed the best in CV above. (Random Forest took quite a long time)

In [123]:
from sklearn.model_selection import train_test_split

In [132]:
X_train.shape

(14640, 300)

In [133]:
X_train, X_val, y_train, y_val = train_test_split(X_train, response, test_size=0.25, random_state=42, stratify=response)

In [158]:
clf= MLPClassifier(alpha=1e-4, hidden_layer_sizes=(6,4), random_state=12,validation_fraction=0)

In [159]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(6, 4), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=12, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0, verbose=False,
       warm_start=False)

In [160]:
val_pred=clf.predict(X_val.toarray())
from sklearn.metrics import accuracy_score

cal_error=accuracy_score(y_val,val_pred)
print(cal_error)

0.809836065574


In [169]:
import pickle
from sklearn.externals import joblib

In [170]:
joblib.dump(clf, 'mlp_classifier.pkl')

['mlp_classifier.pkl']